# Data Loading

In [1]:
import pandas as pd

df = pd.read_csv("../data/Housing.csv")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [3]:
df.isnull().sum()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

In [4]:
df.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [5]:
df.duplicated().sum()

np.int64(0)

In [6]:
def detect_outliers_iqr(df, columns):
    outlier_indices = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)]
        outlier_indices[col] = outliers.index.tolist()
    return outlier_indices

numerical_cols = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
outliers = detect_outliers_iqr(df, numerical_cols)
for col, indices in outliers.items():
    print(f"Outliers in {col}: {len(indices)}")

Outliers in area: 12
Outliers in bedrooms: 12
Outliers in bathrooms: 1
Outliers in stories: 41
Outliers in parking: 12


In [7]:
def cap_outliers_iqr(df, columns):
    df_capped = df.copy()
    
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = max(0, Q1 - 1.5 * IQR)
        upper = Q3 + 1.5 * IQR
        
        # Apply capping
        df_capped[col] = df[col].clip(lower=lower, upper=upper)
        
        print(f"{col}: capped below {lower:.2f} and above {upper:.2f}")
    
    return df_capped
df_capped = cap_outliers_iqr(df, numerical_cols)

area: capped below 0.00 and above 10500.00
bedrooms: capped below 0.50 and above 4.50
bathrooms: capped below 0.00 and above 3.50
stories: capped below 0.00 and above 3.50
parking: capped below 0.00 and above 2.50


# Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

# Scaling and Encoding

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

cat = [col for col in X.columns if X[col].dtype == 'object']
num = [col for col in X.columns if X[col].dtype != 'object']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[num])
X_test_scaled = scaler.transform(X_test[num])

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_ohe = ohe.fit_transform(X_train[cat])
X_test_ohe = ohe.transform(X_test[cat])

ohe_columns = ohe.get_feature_names_out(cat)

X_train_final = pd.DataFrame(X_train_scaled, columns=num, index=X_train.index)
X_test_final = pd.DataFrame(X_test_scaled, columns=num, index=X_test.index)

X_train_ohe_df = pd.DataFrame(X_train_ohe, columns=ohe_columns, index=X_train.index)
X_test_ohe_df = pd.DataFrame(X_test_ohe, columns=ohe_columns, index=X_test.index)

X_train_processed = pd.concat([X_train_final, X_train_ohe_df], axis=1)
X_test_processed = pd.concat([X_test_final, X_test_ohe_df], axis=1)


In [10]:
import os
import joblib

os.makedirs("models", exist_ok=True)

joblib.dump(scaler, "models/scaler.pkl")

joblib.dump(ohe, "models/encoder.pkl")

print("Scaler and encoder saved successfully in the 'models/' directory.")

Scaler and encoder saved successfully in the 'models/' directory.
